In [ ]:
pip install faiss

     |████████████████████████████████| 4.7 MB 4.0 MB/s 


In [ ]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (125 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [ ]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 132 kB 4.2 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219286 sha256=a0811fade9c6f1e77d610d015662146a4668376a04c9566743ecb2f05a48bee9
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATA_PATH = "gdrive/MyDrive"

with open(DATA_PATH +'/mbert_chi_embeddings_229730.pkl','rb') as f:
  embeddings = pickle.load(f)

In [ ]:
print(len(embeddings["translated_ques"]))
print(len(embeddings["translated_ans"]))
print(len(embeddings["ques_embeddings"]))
print(len(embeddings["ans_embeddings"]))

229730
229730
229730
229730


In [ ]:
with open(DATA_PATH +'/lasse_embeddings.pkl','rb') as f1:
  embeddings_to_add = pickle.load(f1)

In [ ]:
print(len(embeddings_to_add["question"]))
print(len(embeddings_to_add["answer"]))
print(len(embeddings_to_add["question_embedding"]))
print(len(embeddings_to_add["answer_embedding"]))

29752
29752
29752
29752


In [ ]:
embeddings["translated_ques"].extend(embeddings_to_add["question"])
embeddings["translated_ans"].extend(embeddings_to_add["answer"])
embeddings["ques_embeddings"].extend(embeddings_to_add["question_embedding"])
embeddings["ans_embeddings"].extend(embeddings_to_add["answer_embedding"])

In [ ]:
print(len(embeddings["translated_ques"]))
print(len(embeddings["translated_ans"]))
print(len(embeddings["ques_embeddings"]))
print(len(embeddings["ans_embeddings"]))

259482
259482
259482
259482


In [ ]:
import numpy as np 
import faiss

In [ ]:
question_embedding = np.array(embeddings["ques_embeddings"])
answer_embedding = np.array(embeddings["ans_embeddings"])
print(question_embedding.shape)
print(answer_embedding.shape)

(259482, 768)
(259482, 768)


In [ ]:
dimension = 768
nlist = 5          # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  
index_question = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)
index_answer = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

In [ ]:
print(index_question.is_trained)   # False
index_question.train(question_embedding)  # train on the database vectors
print(index_question.ntotal)   # 0
index_question.add(question_embedding)   # add the vectors and update the index
print(index_question.is_trained)  # True
print(index_question.ntotal)

False
0
True
259482


In [ ]:
print(index_answer.is_trained)   # False
index_answer.train(answer_embedding)  # train on the database vectors
print(index_answer.ntotal)   # 0
index_answer.add(answer_embedding)   # add the vectors and update the index
print(index_answer.is_trained)  # True
print(index_answer.ntotal)   # 29752

False
0
True
259482


In [ ]:
embedding_by_len = embeddings["translated_ques"].copy()
embedding_by_len.sort(key=len, reverse=True)

n = 15000

index = []

for i in range(n):
  index.append(embeddings["translated_ques"].index(embedding_by_len[i]))

In [ ]:
len(index)

15000

In [ ]:
k = 4

result = []

for i in range(n):
  num = index[i]
  question_query_vector = np.array([embeddings["ques_embeddings"][num]])
  question_distances, question_indices = index_question.search(question_query_vector, k)
  
  answer_query_vector = np.array([embeddings["ans_embeddings"][num]])
  answer_distances, answer_indices = index_answer.search(answer_query_vector, k)
  
  for j in range(k):
    q_index = question_indices[0][j]
    if (q_index != num) and q_index in answer_indices[0]:
      if embeddings["translated_ques"][num] != embeddings["translated_ques"][q_index] and embeddings["translated_ques"][num] != embeddings["translated_ques"][q_index]:
        result.append((num, q_index))

In [ ]:
print(result)

[(119863, 106085), (216601, 144782), (76429, 50727), (134649, 134648), (25722, 71320), (106085, 119863), (204845, 122785), (109292, 27437), (56727, 87154), (188576, 201271), (63750, 128400), (67871, 103747), (71320, 25722), (42950, 120582), (27437, 109292), (29716, 36415), (111444, 111442), (158353, 71521), (83185, 227264), (128400, 63750), (27812, 63143), (76213, 221938), (112332, 147593), (213277, 34433), (124212, 124210), (111333, 117932), (226525, 61752), (123842, 147254), (116947, 141277), (99151, 90164), (85641, 210561), (86829, 17813), (11813, 162147), (126690, 229103), (48267, 48269), (53863, 39970), (136671, 99962), (17588, 191413), (102832, 56721), (20635, 141277), (192499, 194018), (219197, 219376), (227264, 83185), (89504, 89511), (39970, 53863), (176155, 227148), (220849, 218573), (223967, 76792), (223967, 119473), (51242, 88094), (91098, 137737), (196036, 196678), (191120, 191488), (140807, 66457), (91153, 57123), (205665, 210454), (148694, 119782), (143196, 103651), (143

In [ ]:
print(len(result))

61


In [ ]:
print(embeddings["translated_ques"][119863])
print(embeddings["translated_ans"][119863])

sudden deafness and snorting of ears.When you take the medication, your ears are reduced; your ears are less buzzed but remain buzzed for 24 hours, but the symmetrical resonance of your ears is greater; the noise in your ears continues unabated, the night is quiet and the 16th of January of this 16th drug is again improved, the ear is less buzzing, and the noise is still constant and the night is more noisy.There was also some pain in the ears, drips were still in use, and there was pain and itch in the mouth with a tampon.Sleep and dream more.There has been little change since the beginning of the medication on 3 January, and in recent days it has been difficult to sleep and easy to wake up, with cramps and swellings on both the left and left ears, which are more pronounced, with obvious swelling pains, constant squeezing and half-cerebral echoes.The buzzing of the ears has also been constant, and when the morning wakes up, the buzzes will be softer, and the sound of the loudspeaker i

In [ ]:
print(embeddings["translated_ques"][106085])
print(embeddings["translated_ans"][106085])

sudden deafness and snorting of ears.On 13 October, when the left ear began to suffocate, the sound of the left ear was suffocated on 15 October and the sound of the sound of the talk was a little far away. The county examined the polycobalt and nasal bulges and said it was okay. On 16 October, the hearing fell a little too loud to cover its ears, went to the city hospital to say it was deaf, the membrane was fine, the skull was fine, the first day after the treatment was treated with the deaf fluid, the ear was sore, the hearing recovered, the suffocation was so hard to hear the sound of the sound, the last two weeks of the suffocation reduced the lang, the pain in the left ear was sore during the period, the pain in the left ear was sore, the pain in both ears was now deep enough to dig the ear and nothing was seen.The hearing is almost normal for more than two months now, but the ears have been cut off from each other, sleeping in the morning is more obvious, sleeping on the right s

In [ ]:
for i in range(len(result)):
  first_index = result[i][0]
  second_index = result[i][1]

  first_t_q = embeddings["translated_ques"][first_index]
  first_t_a =embeddings["translated_ans"][second_index]
  first_q_embeddings = embeddings["ques_embeddings"][first_index]
  first_a_embeddings = embeddings["ans_embeddings"][second_index]

  second_t_q = embeddings["translated_ques"][second_index]
  second_t_a =embeddings["translated_ans"][first_index]
  second_q_embeddings = embeddings["ques_embeddings"][second_index]
  second_a_embeddings = embeddings["ans_embeddings"][first_index]

  embeddings["translated_ques"].append(first_t_q)
  embeddings["translated_ans"].append(first_t_a)
  embeddings["ques_embeddings"].append(first_q_embeddings)
  embeddings["ans_embeddings"].append(first_a_embeddings)

  embeddings["translated_ques"].append(second_t_q)
  embeddings["translated_ans"].append(second_t_a)
  embeddings["ques_embeddings"].append(second_q_embeddings)
  embeddings["ans_embeddings"].append(second_a_embeddings)

In [ ]:
print(len(embeddings["chi_ques"]))
print(len(embeddings["chi_ans"]))
print(len(embeddings["translated_ques"]))
print(len(embeddings["translated_ans"]))
print(len(embeddings["ques_embeddings"]))
print(len(embeddings["ans_embeddings"]))

229730
229730
259604
259604
259604
259604


In [38]:
pickle_out = open(DATA_PATH + "/Multilingual+Lasse_embeddings.pkl", "wb")
pickle.dump(embeddings, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()